In [1]:
import featuretools as ft

es = ft.demo.load_mock_customer(return_entityset=True)
print(es)

Entityset: transactions
  DataFrames:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 3]
    sessions [Rows: 35, Columns: 5]
    customers [Rows: 5, Columns: 5]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id
    sessions.customer_id -> customers.customer_id


/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/mle-user/mle_projects/mle-mlflow/.venv_m

In [2]:
import featuretools as ft
from woodwork.logical_types import Categorical, PostalCode

data = ft.demo.load_mock_customer()
print(data['customers'])
transactions_df = data["transactions"].merge(data["sessions"]).merge(data["customers"])
print(transactions_df.head(2))
products_df = data["products"]

es = es.add_dataframe(
    dataframe_name="transactions",
    dataframe=transactions_df,
    index="transaction_id",
    time_index="transaction_time",
    logical_types={
        "product_id": Categorical,
        "zip_code": PostalCode,
    },
)
es = es.add_dataframe(
    dataframe_name="products", dataframe=products_df, index="product_id"
)
es = es.add_relationship("products", "product_id", "transactions", "product_id")
print(es)

   customer_id zip_code           join_date   birthday
0            1    60091 2011-04-17 10:48:33 1994-07-18
1            2    13244 2012-04-15 23:31:04 1986-08-18
2            3    13244 2011-08-13 15:42:34 2003-11-21
3            4    60091 2011-04-08 20:08:14 2006-08-15
4            5    60091 2010-07-17 05:27:50 1984-07-28
   transaction_id  session_id    transaction_time product_id  amount   
0              10           1 2014-01-01 00:00:00          5  127.64  \
1               2           1 2014-01-01 00:01:05          2  109.48   

   customer_id   device session_start zip_code           join_date   birthday  
0            2  desktop    2014-01-01    13244 2012-04-15 23:31:04 1986-08-18  
1            2  desktop    2014-01-01    13244 2012-04-15 23:31:04 1986-08-18  
Entityset: transactions
  DataFrames:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 5]
    customers [Rows: 5, Columns: 5]
  Relationships:
    trans

/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/mle-user/mle_projects/mle-mlflow/.venv_m

In [3]:
import featuretools as ft

es = ft.demo.load_mock_customer(return_entityset=True)

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=["count"],
    trans_primitives=["month"],
    max_depth=1,
)
print(feature_matrix)

/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/mle-user/mle_projects/mle-mlflow/.venv_m

            zip_code  COUNT(sessions) MONTH(birthday) MONTH(join_date)
customer_id                                                           
5              60091                6               7                7
4              60091                8               8                4
1              60091                8               7                4
3              13244                6              11                8
2              13244                7               8                4


In [16]:
# Задание 3. Генерация новых признаков с Autofeat
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from autofeat import AutoFeatRegressor
import pandas as pd
import numpy as np

# Загрузка данных
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

autofeat = AutoFeatRegressor(
    max_gb=16,
    transformations=["log", "1+", "sqrt"]
)

X_train_transformed = autofeat.fit_transform(X_train, y_train)
X_test_transformed = autofeat.transform(X_test)

model = LinearRegression()
model.fit(X_train_transformed, y_train)
print("Модель обучена")

prediction = model.predict(X_test_transformed)
print("Предсказания выполнены")
print(prediction)


Модель обучена
Предсказания выполнены
[26.38913799 32.09538287 13.91812859 21.32593238 15.06818433 20.84785056
 17.5999834  16.29599161 22.23431333 19.46080943 17.67625938 17.45552101
  4.84722516 21.26276627 17.43114543 20.95974555 18.0378348   9.50232635
 51.83601578 13.87151486 27.36796413 28.33446407 14.96134201 21.22524486
 16.85484318 16.95674416 20.94972938 11.53030645 17.2644404  18.72753579
 18.15443696 24.76415472 20.40823422 21.89177    15.41372579 16.44329997
 31.47059816 19.9382588  20.30348715 22.06748622 16.00788456 33.68798204
 49.47984595 16.962626   25.24676558 15.74522575 14.58072368 23.8773916
 18.58454943 26.0726805  17.41995283 35.64313041 17.12718288 25.21555371
 46.13556556 25.24815765 15.04938291 31.19210439 24.81819903 19.23128027
 26.7166774  35.69672452 31.80190205 15.16259704 24.66099507 14.10883317
 15.75034184 23.69094099 29.39821957 12.33581499 21.58343898 27.5002117
 11.36887248 16.69441526 21.83071141  6.3764741  19.82855611 46.55108277
 13.60471906 13